# 3. Koncerny
Nyní se podíváme trochu na rozdělení fondů samotných

In [49]:
import pandas as pd
import numpy as np
%matplotlib inline

import PyPDF2
from io import StringIO
import re
import urllib

## Načítání dat
Data od MMR máme zkombinována s daty of MFČR v souboru `vystupy/projekty-bez-popisu.csv` (můžeme použít i `projekty.csv`, ale ty jsou zbytečně velké pro naše potřeby).

In [3]:
df = pd.read_csv('vystupy/projekty-bez-popisu.csv', low_memory=False)

In [4]:
df.shape

(73446, 18)

In [5]:
# nazvy sloupcu, at se to lip filtruje
prog = 'Program'
ico = 'IČO'
dat = 'Datum'
naz = 'Název'
celk = 'Celkové zdroje'
sml = 'Smlouva'
prop = 'Proplaceno'
zad = 'Žadatel'
prfo = 'Právní forma'
prfok = 'Právní forma (kód)'
nuts = 'NUTS4'
esa = 'ESA'
esak = 'ESA (kód)'
zam = 'Zaměstnanců'
kat = 'Kategorie'
ver = 'Veřejná instituce'

In [6]:
df.loc[:, ico] = [str(j).rjust(8, '0') for j in df.loc[:, ico]]

## Seznam

[top 100](http://www.czechtop100.cz/menu/aktualne/100-nejvyznmanejsich-firem-cr.html)

In [7]:
szn = pd.read_excel('http://www.czechtop100.cz/files/download/ke-stazeni/dokumenty/2015-top-100-hlavni-zebricek.xls')

In [27]:
szn.head(20)

,2014,2013,Firma,Tržby 2014,Tržby 2013
0,1,1,"ŠKODA AUTO, a.s.",299318000,243624000
1,2,2,ČEZ a.s.,200850010,216987816
2,3,4,"AGROFERT, a.s.",166772035,151705412
3,4,3,"RWE Supply & Trading CZ, a.s.",146075223,180213863
4,5,5,"UNIPETROL, a.s.",124229000,99415000
5,6,6,FOXCONN CZ s.r.o.,119563335,97305705
6,7,7,"Energetický a průmyslový holding, a.s.",100900000,82876000
7,8,8,"ČEPRO, a.s.",93130227,88606259
8,9,9,"MORAVIA STEEL, a.s.",59479480,60038524
9,10,10,"Eni Česká republika, s.r.o.",50814425,48090924


In [72]:
df.loc[df.loc[:, zad].str.contains('wotan', flags=re.IGNORECASE).replace(np.nan, False), [celk, sml]].sum()/10**6

Celkové zdroje    0.0
Smlouva           0.0
dtype: float64

    MORAVIA STEEL
    http://www.trz.cz/web/trzocel.nsf/link/spolecnosti_skupina_cz
    18050646, 47675896, 47674539, 25830716, 47672081, 48395862, 27768953, 25870807, 42293588, 26955342, 46978496, 29400066, 28262913

    18050646	Třinecké železárny
    47675896    Energetika Třinec
    47674539    Strojírny a stavby Třinec
    25830716	Slévárny Třinec
    47672081    Řetězárna
    48395862	Refrasil
    27768953    VÚHŽ
    25870807    MATERIÁLOVÝ A METALURGICKÝ VÝZKUM
    42293588    Šroubárna Kyjov
    26955342    HŽP
    46978496    Kovárna Viva
    29400066    ŽDB drátovna
    28262913    DALSELV DESIGN

    ms = '18050646, 47675896, 47674539, 25830716, 47672081, 48395862, 27768953, 25870807, 42293588, 26955342, 46978496, 29400066, 28262913'.split(', ')

    msd = df.loc[df.loc[:, ico].isin(ms)].groupby([ico, zad]).sum().loc[:, [celk, sml, prop]]
    (msd.sort_values(by=sml, ascending=False)/10**9).round(2)

## Agroferti
- [minule](https://gist.github.com/kokes/052cc2a63e6366fc92b1)
- Dotyk nedetailuje
- [Hlidaci pes](http://hlidacipes.org/babisovo-eurodotacni-kolecko-agrofert-zada-politici-ano-schvaluji-ministr-financi-kontroluje/)
- wotan? neni v datech, ale Hlidaci pes je tam ma


In [52]:
def getPDFContent(path, pages=[]):
    content = ""
    p = open(path, "rb")
    pdf = PyPDF2.PdfFileReader(p)
    if pages:
        for i in pages:
            content += pdf.getPage(i).extractText() + "\n"
    else:
        numPages = pdf.getNumPages()
        for i in range(numPages):
            content += pdf.getPage(i).extractText() + "\n"
    content = " ".join(content.replace(u"\xa0", " ").strip().split())
    return content

In [53]:
ag_url = 'http://www.agrofert.cz/media/download/7176'
ag_fn, header = urllib.request.urlretrieve(ag_url)

vz = getPDFContent(ag_fn)

In [59]:
#vz = vz.encode('ascii', 'ignore')
agica = re.findall(r"([0-9]{8})", vz)

# agroico = [int(j) for j in np.unique(m)]
#np.shape(dot.loc[dot.ico.isin(agroico),'castka_alok'])

In [63]:
df.loc[df.loc[:, ico].isin(agica), [celk, sml]].sum()/10**9

Celkové zdroje    5.396987
Smlouva           1.607321
dtype: float64

In [80]:
agr = df.loc[df.loc[:, ico].isin(agica), :].groupby([ico, zad]).sum().loc[:, [celk, sml, prop]]
(agr.sort_values(by=sml, ascending=False)/10**6).round(2).head(20)

,,Celkové zdroje,Smlouva,Proplaceno
IČO,Žadatel,,,
49100262,"Lovochemie, a.s.",1886.33,473.64,472.27
60108916,"Synthesia, a.s.",1370.25,390.74,389.37
00011835,"DEZA, a.s.",577.67,154.43,145.40
26872307,PRECHEZA a.s.,335.93,124.32,105.89
27465021,"Fatra, a.s.",343.78,117.92,105.95
60108975,Výzkumný ústav organických syntéz a.s.,215.49,74.54,73.58
14612411,"CS CABOT, spol. s r.o.",112.05,36.53,35.54
46967851,"PENAM, a.s.",54.33,24.75,22.49
25193473,"PUCLICKÁ, s.r.o.",75.81,19.33,19.31
